In [5]:
!kubecolor -n sqlserver get all

NAME                                   READY   STATUS    RESTARTS   AGE
pod/mssqlserver-2022-97999577d-h2n74   1/1     Running   0          40m

NAME                       TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)    AGE
service/mssqlserver-2022   ClusterIP   10.109.219.112   <none>        1433/TCP   12h

NAME                               READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/mssqlserver-2022   1/1     1            1           12h

NAME                                         DESIRED   CURRENT   READY   AGE
replicaset.apps/mssqlserver-2022-97999577d   1         1         1       12h


In [3]:
!kubectl -n sqlserver get deployment -o yaml | yq

apiVersion: v1
items:
  - apiVersion: apps/v1
    kind: Deployment
    metadata:
      annotations:
        deployment.kubernetes.io/revision: "1"
        meta.helm.sh/release-name: mssqlserver-2022
        meta.helm.sh/release-namespace: sqlserver
      creationTimestamp: "2024-02-01T13:22:01Z"
      generation: 1
      labels:
        app: mssqlserver-2022
        app.kubernetes.io/managed-by: Helm
        chart: mssqlserver-2022-1.2.3
        heritage: Helm
        release: mssqlserver-2022
      name: mssqlserver-2022
      namespace: sqlserver
      resourceVersion: "14350"
      uid: 049e9a18-923e-4f71-8d4a-935276a80211
    spec:
      progressDeadlineSeconds: 600
      replicas: 1
      revisionHistoryLimit: 10
      selector:
        matchLabels:
          app: mssqlserver-2022
          release: mssqlserver-2022
      strategy:
        rollingUpdate:
          maxSurge: 25%
          maxUnavailable: 25%
        type: RollingUpdate
      template:
        metadata:
          cr

In [3]:
!kubectl -n sqlserver get service/mssqlserver-2022 -o yaml | yq

apiVersion: v1
kind: Service
metadata:
  annotations:
    meta.helm.sh/release-name: mssqlserver-2022
    meta.helm.sh/release-namespace: sqlserver
  creationTimestamp: "2024-01-29T21:41:33Z"
  labels:
    app: mssqlserver-2022
    app.kubernetes.io/managed-by: Helm
    chart: mssqlserver-2022-1.2.3
    heritage: Helm
    release: mssqlserver-2022
  name: mssqlserver-2022
  namespace: sqlserver
  resourceVersion: "11028"
  uid: 242e5bb0-4d1b-4a4d-858a-4032fb0efad1
spec:
  clusterIP: 10.109.219.112
  clusterIPs:
    - 10.109.219.112
  internalTrafficPolicy: Cluster
  ipFamilies:
    - IPv4
  ipFamilyPolicy: SingleStack
  ports:
    - name: mssql
      port: 1433
      protocol: TCP
      targetPort: mssql
  selector:
    app: mssqlserver-2022
    release: mssqlserver-2022
  sessionAffinity: None
  type: ClusterIP
status:
  loadBalancer: {}


In [2]:
!nmap -p 1433 mssqlserver-2022.sqlserver.svc 

Starting Nmap 7.80 ( https://nmap.org ) at 2024-01-30 07:15 -03
Nmap scan report for mssqlserver-2022.sqlserver.svc (10.109.219.112)
Host is up (0.00021s latency).
rDNS record for 10.109.219.112: mssqlserver-2022.sqlserver.svc.cluster2.xpt

PORT     STATE SERVICE
1433/tcp open  ms-sql-s

Nmap done: 1 IP address (1 host up) scanned in 0.14 seconds


In [5]:
!kubecolor -n sqlserver get secrets

NAME                                     TYPE                 DATA   AGE
mssqlserver-2022-secret                  Opaque               1      113m
sh.helm.release.v1.mssqlserver-2022.v1   helm.sh/release.v1   1      113m


In [4]:
!kubectl -n sqlserver get secrets mssqlserver-2022-secret -o yaml | yq

apiVersion: v1
data:
  sapassword: WkF5UHNNSzlSTTk5U2ZZZmF6Nlc=
kind: Secret
metadata:
  annotations:
    meta.helm.sh/release-name: mssqlserver-2022
    meta.helm.sh/release-namespace: sqlserver
  creationTimestamp: "2024-01-29T17:18:56Z"
  labels:
    app: mssqlserver-2022
    app.kubernetes.io/managed-by: Helm
    chart: mssqlserver-2022-1.2.3
    heritage: Helm
    release: mssqlserver-2022
  name: mssqlserver-2022-secret
  namespace: sqlserver
  resourceVersion: "30307"
  uid: 8b599acf-0a35-461c-95d7-ad3d7a9d2948
type: Opaque


# SA Password Retrieval

In [1]:
!kubectl -n sqlserver get secrets mssqlserver-2022-secret -o jsonpath="{.data.sapassword}" | base64 --decode

2u7XMoSsOhPJsesijZPd

In [8]:
! password=$(kubectl -n sqlserver get secrets mssqlserver-2022-secret -o jsonpath="{.data.sapassword}" | base64 --decode) && \
  \
  \
  kubectl -n sqlserver run mssqlcli --image=mcr.microsoft.com/mssql-tools -ti --restart=Never --rm -- \
        /opt/mssql-tools/bin/sqlcmd -S mssqlserver-2022.sqlserver.svc -U sa -P $password -Q "select @@VERSION;"

                                                                                                                                                                                                                                                                                                            
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Microsoft SQL Server 2022 (RTM-CU11) (KB5032679) - 16.0.4105.2 (X64) 
	Nov 14 2023 18:33:19 
	Copyright (C) 2022 Microsoft Corporation
	Developer Edition (64-bit) on Linux (Ubuntu 22.04.3 LTS) <X64>                                                                                                      

(1 rows affected)
pod "mssqlcli" deleted


In [ ]:
DBMS: Microsoft SQL Server (ver. 2022.0)
Case sensitivity: plain=mixed, delimited=exact
Unable to find remote host or port in the URL.
